### Step 4 - Generate Results (Automated)

This script is a fully automated version of the previous script (Step 4). It has precisely the same mechanics. As such, to learn how it works, go through the non-automated version first - and then approach this version. 

The only difference / complexity is the use of settings dictionaries to run scenarios in bulk in loop fashion - see the end of the script for examples of this.  

In [1]:
import pandas as pd
import os, sys
sys.path.append(r'C:\Users\charl\Documents\GitHub\GOST_PublicGoods\GOSTNets\GOSTNets')
sys.path.append(r'C:\Users\charl\Documents\GitHub\GOST')
import GOSTnet as gn
import importlib
import geopandas as gpd
import rasterio as rt
from rasterio import features
from shapely.wkt import loads
import numpy as np
import networkx as nx
from shapely.geometry import box, Point, Polygon
import warnings 

peartree version: 0.6.1 
networkx version: 2.3 
matplotlib version: 3.0.3 
osmnx version: 0.9 


### Global Settings

In [2]:
pth = r'C:\Users\charl\Documents\GOST\Yemen\graphtool'
basepth = r'C:\Users\charl\Documents\GOST\Yemen'
util_path = r'C:\Users\charl\Documents\GOST\Yemen\util_files'
srtm_pth = r'C:\Users\charl\Documents\GOST\Yemen\SRTM'
services = ['ALL',
            'Antenatal',
            'BEmONC',
            'CEmONC',
            'Under_5',
            'Emergency_Surgery',
            'Immunizations',
            'Malnutrition',
            'Int_Outreach']

### Define function to add elevation to a point GeoDataFrame

In [3]:
def add_elevation(df, x, y, srtm_pth):
    # walk all tiles, find path
    
    tiles = []
    for root, folder, files in os.walk(os.path.join(srtm_pth,'high_res')):
        for f in files:
            if f[-3:] == 'hgt':
                tiles.append(f[:-4])

    # load dictionary of tiles
    arrs = {}
    for t in tiles:
        arrs[t] = rt.open(srtm_pth+r'\high_res\{}.hgt\{}.hgt'.format(t, t), 'r')

    # assign a code
    uniques = []
    df['code'] = 'placeholder'
    def tile_code(z):
        E = str(z[x])[:2]
        N = str(z[y])[:2]
        return 'N{}E0{}'.format(N, E)
    df['code'] = df.apply(lambda z: tile_code(z), axis = 1)
    unique_codes = list(set(df['code'].unique()))
    
    z = {}
    # Match on High Precision Elevation
    property_name = 'elevation'
    for code in unique_codes:
        
        df2 = df.copy()
        df2 = df2.loc[df2['code'] == code]
        dataset = arrs[code]
        b = dataset.bounds
        datasetBoundary = box(b[0], b[1], b[2], b[3])
        selKeys = []
        selPts = []
        for index, row in df2.iterrows():
            if Point(row[x], row[y]).intersects(datasetBoundary):
                selPts.append((row[x],row[y]))
                selKeys.append(index)
        raster_values = list(dataset.sample(selPts))
        raster_values = [x[0] for x in raster_values]

        # generate new dictionary of {node ID: raster values}
        z.update(zip(selKeys, raster_values))
        
    elev_df = pd.DataFrame.from_dict(z, orient='index')
    elev_df.columns = ['elevation']
    
    missing = elev_df.copy()
    missing = missing.loc[missing.elevation < 0]
    if len(missing) > 0:
        missing_df = df.copy()
        missing_df = missing_df.loc[missing.index]
        low_res_tifpath = os.path.join(srtm_pth, 'clipped', 'clipped_e20N40.tif')
        dataset = rt.open(low_res_tifpath, 'r')
        b = dataset.bounds
        datasetBoundary = box(b[0], b[1], b[2], b[3])
        selKeys = []
        selPts = []
        for index, row in missing_df.iterrows():
            if Point(row[x], row[y]).intersects(datasetBoundary):
                selPts.append((row[x],row[y]))
                selKeys.append(index)
        raster_values = list(dataset.sample(selPts))
        raster_values = [x[0] for x in raster_values]
        z.update(zip(selKeys, raster_values))

        elev_df = pd.DataFrame.from_dict(z, orient='index')
        elev_df.columns = ['elevation']
    df['point_elev'] = elev_df['elevation']
    df = df.drop('code', axis = 1)
    return df

### Define function to convert distances to walk times

In [4]:
def generate_walktimes(df, start = 'point_elev', end = 'node_elev', dist = 'NN_dist', max_walkspeed = 6, min_speed = 0.1):
    # Tobler's hiking function: https://en.wikipedia.org/wiki/Tobler%27s_hiking_function
    def speed(incline_ratio, max_speed):
        walkspeed = max_speed * np.exp(-3.5 * abs(incline_ratio + 0.05)) 
        return walkspeed

    speeds = {}
    times = {}

    for index, data in df.iterrows():
        if data[dist] > 0:
            delta_elevation = data[end] - data[start]
            incline_ratio = delta_elevation / data[dist]
            speed_kmph = speed(incline_ratio = incline_ratio, max_speed = max_walkspeed)
            speed_kmph = max(speed_kmph, min_speed)
            speeds[index] = (speed_kmph)
            times[index] = (data[dist] / 1000 * 3600 / speed_kmph)

    speed_df = pd.DataFrame.from_dict(speeds, orient = 'index')
    time_df = pd.DataFrame.from_dict(times, orient = 'index')

    df['walkspeed'] = speed_df[0]
    df['walk_time'] = time_df[0]
    
    return df

In [5]:
# Intersect points with merged districts shapefile, identify relationship

def AggressiveSpatialIntersect(points, polygons):
    import osmnx as ox
    spatial_index = points.sindex
    container = {}
    cut_geoms = []
    for index, row in polygons.iterrows():
        polygon = row.geometry
        if polygon.area > 0.5:
            geometry_cut = ox.quadrat_cut_geometry(polygon, quadrat_width=0.5)
            cut_geoms.append(geometry_cut)
            print('cutting geometry %s into %s pieces' % (index, len(geometry_cut)))
            index_list = []
            for P in geometry_cut:
                possible_matches_index = list(spatial_index.intersection(P.bounds))
                possible_matches = points.iloc[possible_matches_index]
                precise_matches = possible_matches[possible_matches.intersects(P)]
                if len(precise_matches) > 0:
                    index_list.append(precise_matches.index)
                flat_list = [item for sublist in index_list for item in sublist]
                container[index] = list(set(flat_list))
        else:
            possible_matches_index = list(spatial_index.intersection(polygon.bounds))
            possible_matches = points.iloc[possible_matches_index]
            precise_matches = possible_matches[possible_matches.intersects(polygon)]
            if len(precise_matches) > 0:
                container[index] = list(precise_matches.index)
    return container

### Generate Zonal Stats

In [6]:
def zonalStats(inShp, inRaster, bandNum=1, mask_A = None, reProj = False, minVal = '', maxVal = '', verbose=False , rastType='N', unqVals=[]):
    import sys, os, inspect, logging, json
    import rasterio, affine

    import pandas as pd
    import geopandas as gpd
    import numpy as np

    from collections import Counter
    from shapely.geometry import box
    from affine import Affine
    from rasterio import features
    from rasterio.mask import mask
    from rasterio.features import rasterize
    from rasterio.warp import reproject, Resampling
    from osgeo import gdal
    
    ''' Run zonal statistics against an input shapefile
    
    INPUT VARIABLES
    inShp [string or geopandas object] - path to input shapefile
    inRaster [string or rasterio object] - path to input raster
    
    OPTIONAL
    bandNum [integer] - band in raster to analyze
    reProj [boolean] -  whether to reproject data to match, if not, raise an error
    minVal [number] - if defined, will only calculation statistics on values above this number
    verbose [boolean] - whether to be loud with responses
    rastType [string N or C] - N is numeric and C is categorical. Categorical returns counts of numbers
    unqVals [array of numbers] - used in categorical zonal statistics, tabulates all these numbers, will report 0 counts
    mask_A [numpy boolean mask] - mask the desired band using an identical shape boolean mask. Useful for doing conditional zonal stats
    
    RETURNS
    array of arrays, one for each feature in inShp
    '''   
    if isinstance(inShp, str):
        inVector = gpd.read_file(inShp) 
    else:
        inVector = inShp
    if isinstance(inRaster, str):
        curRaster = rasterio.open(inRaster, 'r+')
    else:
        curRaster = inRaster
        
    # If mask is not none, apply mask 
    if mask_A is not None:
        
        curRaster.write_mask(np.invert(mask_A))
    
    outputData=[]
    if inVector.crs != curRaster.crs:
        if reProj:
            inVector = inVector.to_crs(curRaster.crs)
        else:
            raise ValueError("Input CRS do not match")
    fCount = 0
    tCount = len(inVector['geometry'])
    #generate bounding box geometry for raster bbox
    b = curRaster.bounds
    rBox = box(b[0], b[1], b[2], b[3])
    for geometry in inVector['geometry']:
        #This test is used in case the geometry extends beyond the edge of the raster
        #   I think it is computationally heavy, but I don't know of an easier way to do it
        if not rBox.contains(geometry):
            geometry = geometry.intersection(rBox)            
        try:
            fCount = fCount + 1
            if fCount % 1000 == 0 and verbose:
                tPrint("Processing %s of %s" % (fCount, tCount) )
            # get pixel coordinates of the geometry's bounding box
            ul = curRaster.index(*geometry.bounds[0:2])
            lr = curRaster.index(*geometry.bounds[2:4])
            '''
            TODO: There is a problem with the indexing - if the shape falls outside the boundaries, it errors
                I want to change it to just grab what it can find, but my brain is wrecked and I cannot figure it out
            print(geometry.bounds)
            print(curRaster.shape)
            print(lr)
            print(ul)
            lr = (max(lr[0], 0), min(lr[1], curRaster.shape[1]))
            ul = (min(ul[0], curRaster.shape[0]), min(ul[1]))
            '''
            # read the subset of the data into a numpy array
            window = ((float(lr[0]), float(ul[0]+1)), (float(ul[1]), float(lr[1]+1)))
            
            if mask is not None:
                data = curRaster.read(bandNum, window=window, masked = True)
            else:
                data = curRaster.read(bandNum, window=window, masked = False)
            
            # create an affine transform for the subset data
            t = curRaster.transform
            shifted_affine = Affine(t.a, t.b, t.c+ul[1]*t.a, t.d, t.e, t.f+lr[0]*t.e)

            # rasterize the geometry
            mask = rasterize(
                [(geometry, 0)],
                out_shape=data.shape,
                transform=shifted_affine,
                fill=1,
                all_touched=False,
                dtype=np.uint8)

            # create a masked numpy array
            masked_data = np.ma.array(data=data, mask=mask.astype(bool))
            if rastType == 'N':                
                if minVal != '' or maxVal != '':
                    if minVal != '':
                        masked_data = np.ma.masked_where(masked_data < minVal, masked_data)
                    if maxVal != '':
                        masked_data = np.ma.masked_where(masked_data > maxVal, masked_data)                    
                    if masked_data.count() > 0:                        
                        results = [masked_data.sum(), masked_data.min(), masked_data.max(), masked_data.mean()]
                    else :
                        results = [-1, -1, -1, -1]                
                else:
                    results = [masked_data.sum(), masked_data.min(), masked_data.max(), masked_data.mean()]
            if rastType == 'C':
                if len(unqVals) > 0:                          
                    xx = dict(Counter(data.flatten()))
                    results = [xx.get(i, 0) for i in unqVals]                
                else:
                    results = np.unique(masked_data, return_counts=True)                    
            outputData.append(results)
        except Exception as e: 
            print(e)
            outputData.append([-1, -1, -1, -1])            
    return outputData   

### Define Main Process

In [7]:
def Process(settings_dict):
    
    print(settings_dict)
    walking = settings_dict['walking']
    conflict = settings_dict['conflict']
    zonal_stats = settings_dict['zonal_stats']
    facility_type = settings_dict['facility_type']
    year = settings_dict['year']
    service_index = settings_dict['service_index']
    path_mod = settings_dict['path_mod']
    warfronts = settings_dict['warfronts']
    
    if walking == 1:
        type_tag = 'walking'
        net_name = r'walk_graph.pickle'
    else:
        type_tag = 'driving'
        net_name = r'G_salty_time_conflict_adj.pickle'

    if conflict == 1: 
        conflict_tag = 'ConflictAdj'
        OD_name = r'OD_Jan24th_%s_%s.csv' % (type_tag, year)
    else:
        conflict_tag = 'NoConflict'
        OD_name = r'OD_normal_%s_%s.csv' % (type_tag, year)

    OD_pth = pth
    net_pth = pth

    WGS = {'init':'epsg:4326'}
    measure_crs = {'init':'epsg:32638'}

    subset = r'%s_HERAMS_%s_%s_%s_%s' % (type_tag, facility_type, services[service_index], conflict_tag, year)
    
    if warfronts == 0 and conflict == 1:
        subset = subset+'_no_warfronts'
        
    print("Output files will have name: ", subset)
    print("network: ",net_name)
    print("OD Matrix: ",OD_name)
    print("Conflict setting: ",conflict_tag)

    offroad_speed = 4
    
    OD = pd.read_csv(os.path.join(OD_pth, OD_name))
    OD = OD.rename(columns = {'Unnamed: 0':'O_ID'})
    OD = OD.set_index('O_ID')
    OD = OD.replace([np.inf, -np.inf], np.nan)
    OD_original = OD.copy()
    
    
    ### Optional: Subset to Accepted Nodes
    
    acceptable_df = pd.read_csv(os.path.join(OD_pth, 'HeRAMS 2018 April_snapped.csv'))    

    # Adjust for facility type
    if facility_type == 'HOS':
        acceptable_df = acceptable_df.loc[acceptable_df['Health Facility Type Coded'].isin(['1',1])]
    elif facility_type == 'PHC':
        acceptable_df = acceptable_df.loc[acceptable_df['Health Facility Type Coded'].isin([2,'2',3,'3'])]
    elif facility_type == 'ALL':
        pass
    else:
        raise ValueError('unacceptable facility_type entry!')

    # Adjust for functionality in a given year
    acceptable_df = acceptable_df.loc[acceptable_df['Functioning %s' % year].isin(['1','2',1,2])]
    
    Yemn_bound = gpd.read_file(os.path.join(util_path, 'Yemen_bound.shp'))
    Yemn_bound = Yemn_bound.to_crs(WGS)
    Yemn_bound = Yemn_bound.geometry.iloc[0]

    # Adjust for availability of service

    SERVICE_DICT = {'Antenatal_2018':'ANC 2018',
                   'Antenatal_2016':'Antenatal Care (P422) 2016',
                   'BEmONC_2018':'Basic emergency obstetric care 2018',
                   'BEmONC_2016':'Basic Emergency Obsteteric Care (P424) 2016',
                   'CEmONC_2018':'Comprehensive emergency obstetric care 2018',
                   'CEmONC_2016':'Comprehensive Emergency Obstetric Care (S424) 2016',
                   'Under_5_2018':'Under 5 clinics 2018',
                   'Under_5_2016':'Under-5 clinic services (P23) 2016',
                   'Emergency_Surgery_2018':'Emergency and elective surgery 2018',
                   'Emergency_Surgery_2016':'Emergency and Elective Surgery (S14) 2016',
                   'Immunizations_2018':'EPI 2018',
                   'Immunizations_2016':'EPI (P21a) 2016',
                   'Malnutrition_2018':'Malnutrition services 2018',
                   'Malnutrition_2016':'Malnutrition services (P25) 2016',
                   'Int_Outreach_2018':'Integrated outreach (IMCI+EPI+ANC+Nutrition_Services) 2018',
                   'Int_Outreach_2016':'Integrated Outreach (P22) 2016'}

    if service_index == 0:
        pass
    else:
        acceptable_df = acceptable_df.loc[acceptable_df[SERVICE_DICT['%s_%s' % (services[service_index],year)]].isin(['1',1])]

    len(acceptable_df)
    
    acceptable_df['geometry'] = acceptable_df['geometry'].apply(loads)
    acceptable_gdf = gpd.GeoDataFrame(acceptable_df, geometry = 'geometry', crs = {'init':'epsg:4326'})
    acceptable_gdf = acceptable_gdf.loc[acceptable_gdf['geometry'].intersects(Yemn_bound)]
    accepted_facilities = list(set(list(acceptable_gdf.NN)))
    accepted_facilities_str = [str(i) for i in accepted_facilities]
    OD = OD_original[accepted_facilities_str]
    acceptable_df.to_csv(os.path.join(basepth,'output_layers','Round 3',path_mod,'%s.csv' % subset))
    
    ### Add elevation for destination nodes
    
    dest_df = acceptable_df[['NN','NN_dist','Latitude','Longitude']]
    dest_df = add_elevation(dest_df, 'Longitude','Latitude', srtm_pth).set_index('NN')
    
    ### Add elevation from graph nodes (reference)
    
    G = nx.read_gpickle(os.path.join(OD_pth, net_name))
    G_node_df = gn.node_gdf_from_graph(G)
    G_node_df = add_elevation(G_node_df, 'x', 'y', srtm_pth)
    match_node_elevs = G_node_df[['node_ID','point_elev']].set_index('node_ID')
    match_node_elevs.loc[match_node_elevs.point_elev < 0] = 0
    
    
    ### Match on node elevations for dest_df; calculate travel times to nearest node
    dest_df['node_elev'] = match_node_elevs['point_elev']
    dest_df = generate_walktimes(dest_df, start = 'node_elev', end = 'point_elev', dist = 'NN_dist', max_walkspeed = offroad_speed)
    dest_df = dest_df.sort_values(by = 'walk_time', ascending = False)
    
    
    ### Add Walk Time to all travel times in OD matrix
    
    dest_df = dest_df[['walk_time']]
    dest_df.index = dest_df.index.map(str)

    d_f = OD.transpose()

    for i in d_f.columns:
        dest_df[i] = d_f[i]

    for i in dest_df.columns:
        if i == 'walk_time':
            pass
        else:
            dest_df[i] = dest_df[i] + dest_df['walk_time']

    dest_df = dest_df.drop('walk_time', axis = 1)

    dest_df = dest_df.transpose()
    
    
    ### Import Shapefile Describing Regions of Control
    
    if warfronts == 1:
        conflict_file = r'merged_dists.shp'
    elif warfronts == 0:
        conflict_file = r'NoConflict.shp'
    merged_dists = gpd.read_file(os.path.join(util_path, conflict_file))
    if merged_dists.crs != {'init':'epsg:4326'}:
        merged_dists = merged_dists.to_crs({'init':'epsg:4326'})
    merged_dists = merged_dists.loc[merged_dists.geometry.type == 'Polygon']
    
    ### Factor in lines of Control - Import Areas of Control Shapefile
    
    graph_node_gdf = gn.node_gdf_from_graph(G)
    gdf = graph_node_gdf.copy()
    gdf = gdf.set_index('node_ID')
    possible_snap_nodes = AggressiveSpatialIntersect(graph_node_gdf, merged_dists)
    print('**bag of possible node snapping locations has been successfully generated**')

    
    # Match on network time from origin node (time travelling along network + walking to destination)
    
    ### Load Grid
    if year == 2018:
        year_raster = 2018
    elif year == 2016:
        year_raster = 2015
    grid_name = r'origins_1km_%s_snapped.csv' % year_raster
    grid = pd.read_csv(os.path.join(OD_pth, grid_name))
    grid = grid.rename({'Unnamed: 0':'PointID'}, axis = 1)
    grid['geometry'] = grid['geometry'].apply(loads)
    grid_gdf = gpd.GeoDataFrame(grid, crs = WGS, geometry = 'geometry')
    grid_gdf = grid_gdf.set_index('PointID')
    
    ### Adjust Nearest Node snapping for War
    
    origin_container = AggressiveSpatialIntersect(grid_gdf, merged_dists)
    print('**bag of possible origins locations has been successfully generated**')

    bundle = []
    for key in origin_container.keys():
        origins = origin_container[key]
        possible_nodes = graph_node_gdf.loc[possible_snap_nodes[key]]
        origin_subset = grid_gdf.loc[origins]
        origin_subset_snapped = gn.pandana_snap_points(origin_subset, 
                                    possible_nodes, 
                                    source_crs = 'epsg:4326', 
                                    target_crs = 'epsg:32638', 
                                    add_dist_to_node_col = True)
        bundle.append(origin_subset_snapped)

    grid_gdf_adjusted = pd.concat(bundle)
    
    grid_gdf = grid_gdf_adjusted
    
    ### Adjust acceptable destinations for each node for the war
    
    gdf = graph_node_gdf.copy()
    gdf['node_ID'] = gdf['node_ID'].astype('str')
    gdf = gdf.loc[gdf.node_ID.isin(list(dest_df.columns))]
    gdf = gdf.set_index('node_ID')

    dest_container = AggressiveSpatialIntersect(gdf, merged_dists)

    gdf = graph_node_gdf.copy()
    gdf = gdf.loc[gdf.node_ID.isin(list(dest_df.index))]
    gdf = gdf.set_index('node_ID')

    origin_snap_container = AggressiveSpatialIntersect(gdf, merged_dists)

    bundle = []
    for key in origin_snap_container.keys():
        origins = origin_snap_container[key]
        destinations = dest_container[key]
        Q = dest_df[destinations].loc[origins]
        Q['min_time'] = Q.min(axis = 1)
        Q2 = Q[['min_time']]
        bundle.append(Q2)
    Q3 = pd.concat(bundle)

    dest_df['min_time'] = Q3['min_time']
    
    ### Return to Normal Process
    grid_gdf = grid_gdf.rename(columns = {'NN':'O_ID'})
    
    ### Merge on min Time
    
    grid_gdf = grid_gdf.reset_index()
    grid_gdf = grid_gdf.set_index(grid_gdf['O_ID'])
    grid_gdf['on_network_time'] = dest_df['min_time']
    grid_gdf = grid_gdf.set_index('PointID')
    
    
    # Add origin node distance to network - walking time
    grid = grid_gdf
    grid = add_elevation(grid, 'Longitude','Latitude', srtm_pth)
    grid = grid.reset_index()
    grid = grid.set_index('O_ID')
    grid['node_elev'] = match_node_elevs['point_elev']
    grid = grid.set_index('PointID')
    grid = generate_walktimes(grid, start = 'point_elev', end = 'node_elev', dist = 'NN_dist', max_walkspeed = offroad_speed)
    grid = grid.rename({'node_elev':'nr_node_on_net_elev', 
                        'walkspeed':'walkspeed_to_net', 
                        'walk_time':'walk_time_to_net',
                       'NN_dist':'NN_dist_to_net'}, axis = 1)
    grid['total_time_net'] = grid['on_network_time'] + grid['walk_time_to_net']
    
    ### Calculate Direct Walking Time (not using road network), vs. network Time
    
    bundle = []
    W = graph_node_gdf.copy()
    W['node_ID'] = W['node_ID'].astype(str)
    W = W.set_index('node_ID')

    locations_gdf = gpd.GeoDataFrame(acceptable_df, geometry = 'geometry', crs = {'init':'epsg:4326'})
    locations_container = AggressiveSpatialIntersect(locations_gdf, merged_dists)

    for key in origin_container.keys():
        origins = origin_container[key]
        origin_subset = grid.copy()
        origin_subset = origin_subset.loc[origins]
        locations = locations_gdf.loc[locations_container[key]]
        if len(locations) < 1:
            origin_subset['NN'] = None
            origin_subset['NN_dist'] = None
            bundle.append(origin_subset)
        else:
            origin_subset_snapped = gn.pandana_snap_points(origin_subset, 
                                    locations, 
                                    source_crs = 'epsg:4326', 
                                    target_crs = 'epsg:32638', 
                                    add_dist_to_node_col = True)
            bundle.append(origin_subset_snapped)

    grid_gdf_adjusted = pd.concat(bundle)
    
    grid = grid_gdf_adjusted
    
    Y = grid.copy()
    objs = []
    if len(Y.loc[Y['NN'].isnull() == True]) > 0:
        Y2 = Y.loc[Y['NN'].isnull() == True]
        Y2['walkspeed_direct'] = 0
        Y2['walk_time_direct'] = 9999999
        Y2['NN_dist_direct'] = 9999999
        objs.append(Y2)

    location_elevs = add_elevation(locations_gdf, 'Longitude','Latitude', srtm_pth)
    Y1 = Y.loc[Y['NN'].isnull() == False]
    Y1['NN'] = Y1['NN'].astype(int)
    Y1 = Y1.set_index('NN')
    Y1['dest_NN_elev'] = location_elevs['point_elev']

    Y1 = Y1.reset_index()
    Y1 = generate_walktimes(Y1, start = 'point_elev', end = 'dest_NN_elev', dist = 'NN_dist', max_walkspeed = offroad_speed).reset_index()
    Y1 = Y1.rename({'walkspeed':'walkspeed_direct', 
                        'walk_time':'walk_time_direct',
                       'NN_dist':'NN_dist_direct'}, axis = 1)
    objs.append(Y1)

    grid = pd.concat(objs)

    grid['PLOT_TIME_SECS'] = grid[['walk_time_direct','total_time_net']].min(axis = 1)
    grid['PLOT_TIME_MINS'] = grid['PLOT_TIME_SECS'] / 60
    
    ### Burn Raster
    if year == 2018:
        yr = 18
    elif year == 2016:
        yr = 15
    
    rst_fn = os.path.join(pth,'pop18_resampled_clipped.tif')
    out_fn = os.path.join(basepth,'output_layers','Round 3',path_mod,'%s.tif' % subset)

    # Update metadata
    rst = rt.open(rst_fn, 'r')
    meta = rst.meta.copy()
    D_type = rt.float64
    meta.update(compress='lzw', dtype = D_type, count = 2)

    with rt.open(out_fn, 'w', **meta) as out:
        with rt.open(rst_fn, 'r') as pop:

            # this is where we create a generator of geom, value pairs to use in rasterizing
            shapes = ((geom,value) for geom, value in zip(grid.geometry, grid.PLOT_TIME_MINS))

            population = pop.read(1).astype(D_type)
            cpy = population.copy()

            travel_times = features.rasterize(shapes=shapes, fill=0, out=cpy, transform=out.transform)

            out.write_band(1, population)
            out.write_band(2, travel_times)
    
    if zonal_stats == 0:
        pass
    else:
        for resolution in ['national','district']:
            out_fn = os.path.join(r'C:\Users\charl\Documents\GOST\Yemen\output_layers','Round 3',path_mod,'%s.tif' % subset)

            sys.path.append(r'C:\Users\charl\Documents\GitHub\GOST\GOSTRocks')

            utils = r'C:\Users\charl\Documents\GOST\Yemen\util_files'

            yemen_shp_name = os.path.join(utils, r'Yemen_bound.shp')
            yemen_shp = gpd.read_file(yemen_shp_name)

            if yemen_shp.crs != {'init': 'epsg:4326'}:
                yemen_shp = yemen_shp.to_crs({'init': 'epsg:4326'})

            district_shp_name = os.path.join(r'C:\Users\charl\Documents\GOST\Yemen\VulnerabilityMatrix', r'VM.shp')
            district_shp = gpd.read_file(district_shp_name)

            if district_shp.crs != {'init': 'epsg:4326'}:
                district_shp = district_shp.to_crs({'init': 'epsg:4326'})

            inraster = out_fn
            ras = rt.open(inraster, mode = 'r+')
            pop = ras.read(1)
            tt_matrix = ras.read(2)

            if resolution == 'national':
                target_shp = yemen_shp
            elif resolution == 'district':
                target_shp = district_shp

            ## First, add on the total population of the district to each district shape

            mask_pop = np.ma.masked_where(pop > (200000), pop).mask

            base_pop = zonalStats(target_shp, 
                                    inraster, 
                                    bandNum = 1,
                                    mask_A = mask_pop,
                                    reProj = False, 
                                    minVal = 0,
                                    maxVal = np.inf, 
                                    verbose = True, 
                                    rastType='N')

            cols = ['total_pop','min','max','mean']

            temp_df = pd.DataFrame(base_pop, columns = cols)

            target_shp['total_pop'] = temp_df['total_pop']
            target_shp['total_pop'].loc[target_shp['total_pop'] == -1] = 0

            ## Now, calculate the population within a range of time thresholds from the destination set
            for time_thresh in [30,60,120, 240]:

                mask_obj = np.ma.masked_where(tt_matrix > (time_thresh), tt_matrix).mask

                raw = zonalStats(target_shp, 
                                    inraster, 
                                    bandNum = 1,
                                    mask_A = mask_obj,
                                    reProj = False, 
                                    minVal = 0,
                                    maxVal = np.inf, 
                                    verbose = True, 
                                    rastType='N')

                cols = ['pop_%s' % time_thresh,'min','max','mean']

                temp_df = pd.DataFrame(raw, columns = cols)

                target_shp['pop_%s' % time_thresh] = temp_df['pop_%s' % time_thresh]
                target_shp['pop_%s' % time_thresh].loc[target_shp['pop_%s' % time_thresh] == -1] = 0
                target_shp['frac_%s' % time_thresh] = (target_shp['pop_%s' % time_thresh]) / (target_shp['total_pop']).fillna(0)
                target_shp['frac_%s' % time_thresh].replace([np.inf, -np.inf], 0)
                target_shp['frac_%s' % time_thresh] = target_shp['frac_%s' % time_thresh].fillna(0)

            # Save to file

            if resolution == 'national':
                print('saving national')
                outter = target_shp[['total_pop','pop_30','frac_30','pop_60','frac_60','pop_120','frac_120','pop_240','frac_240']]
                outter.to_csv(os.path.join(basepth, 'output_layers','Round 3',path_mod,'%s_zonal_%s.csv'% (subset, resolution)))
            else:
                print('saving district')
                target_shp['abs_pop_iso'] = target_shp['total_pop'] - target_shp['pop_30']
                target_shp.to_file(os.path.join(basepth, 'output_layers','Round 3',path_mod,'%s_zonal_%s.shp' % (subset, resolution)), driver = 'ESRI Shapefile')


    print('\n**process complete**')

### Execution

In [8]:
new_global_bundle = [
         {'walking':0,'conflict':0,'zonal_stats':1,'facility_type':'HOS','year':2018,'service_index':0,'path_mod':'global','warfronts':0},
         {'walking':0,'conflict':0,'zonal_stats':1,'facility_type':'HOS','year':2016,'service_index':0,'path_mod':'global','warfronts':0},
         {'walking':0,'conflict':0,'zonal_stats':1,'facility_type':'PHC','year':2018,'service_index':0,'path_mod':'global','warfronts':0},          
         {'walking':0,'conflict':0,'zonal_stats':1,'facility_type':'PHC','year':2016,'service_index':0,'path_mod':'global','warfronts':0},
         {'walking':0,'conflict':1,'zonal_stats':1,'facility_type':'HOS','year':2018,'service_index':0,'path_mod':'global','warfronts':1},
         {'walking':0,'conflict':1,'zonal_stats':1,'facility_type':'PHC','year':2018,'service_index':0,'path_mod':'global','warfronts':1},
         {'walking':0,'conflict':1,'zonal_stats':1,'facility_type':'HOS','year':2018,'service_index':0,'path_mod':'global','warfronts':0},
         {'walking':0,'conflict':1,'zonal_stats':1,'facility_type':'PHC','year':2018,'service_index':0,'path_mod':'global','warfronts':0}]

In [9]:
service_2018_conflict_bundle = []
for i in range(1,9):
    S = {'walking':0,        
          'conflict':1,
          'zonal_stats':1,
          'facility_type':'ALL',
          'year':2018,
          'service_index':i,
          'path_mod':'Services_2018_conflictadj',
          'warfronts':1}
    service_2018_conflict_bundle.append(S)

service_2018_noconflict_bundle = []
for i in range(1,9):
    S = {'walking':0,        
          'conflict':0,
          'zonal_stats':1,
          'facility_type':'ALL',
          'year':2018,
          'service_index':i,
          'path_mod':'Services_2018_noconflict',
          'warfronts':0}
    service_2018_noconflict_bundle.append(S)
    
service_2016_bundle = []
for i in range(1,9):
    S = {'walking':0,        
          'conflict':0,
          'zonal_stats':1,
          'facility_type':'ALL',
          'year':2016,
          'service_index':i,
          'path_mod':'Services_2016_noconflict',
          'warfronts':0}
    service_2016_bundle.append(S)

In [13]:
warnings.simplefilter('ignore')
for bundle in [new_global_bundle[4],new_global_bundle[5]]:
    Process(bundle)

{'walking': 0, 'conflict': 1, 'zonal_stats': 1, 'facility_type': 'HOS', 'year': 2018, 'service_index': 0, 'path_mod': 'global', 'warfronts': 1}
Output files will have name:  driving_HERAMS_HOS_ALL_ConflictAdj_2018
network:  G_salty_time_conflict_adj.pickle
OD Matrix:  OD_Jan24th_driving_2018.csv
Conflict setting:  ConflictAdj
cutting geometry 56 into 121 pieces
cutting geometry 59 into 47 pieces
cutting geometry 78 into 236 pieces
cutting geometry 79 into 16 pieces
**bag of possible node snapping locations has been successfully generated**
cutting geometry 56 into 121 pieces
cutting geometry 59 into 47 pieces
cutting geometry 78 into 236 pieces
cutting geometry 79 into 16 pieces
**bag of possible origins locations has been successfully generated**
cutting geometry 56 into 121 pieces
cutting geometry 59 into 47 pieces
cutting geometry 78 into 236 pieces
cutting geometry 79 into 16 pieces
cutting geometry 56 into 121 pieces
cutting geometry 59 into 47 pieces
cutting geometry 78 into 236 

In [ ]:
warnings.simplefilter('ignore')
for bundle in service_2018_conflict_bundle:
    Process(bundle)

In [ ]:
warnings.simplefilter('ignore')
for bundle in service_2018_noconflict_bundle:
    Process(bundle)

In [ ]:
warnings.simplefilter('ignore')
for bundle in service_2016_bundle:
    Process(bundle)